# Bubble and Quick sort

## Utility

You would to skip it first.

In [361]:
import random
import operator
import timeit


idx = operator.itemgetter(0)
val = operator.itemgetter(1)


def pairs(arr):
    return zip(enumerate(arr), enumerate(arr[1:], 1))


def is_bad(pair):
    a, b = pair
    return val(a) > val(b)


def bad_pairs(arr):
    return filter(is_bad, pairs(arr))


def is_sorted(arr):
    return not any(bad_pairs(arr))


def swap(arr, a, b):
    arr[b], arr[a] = arr[a], arr[b]


def test_array():
    return [random.randint(0, 100) for _ in range(25)]


def time(sorting_function):
    return timeit.timeit(f'{sorting_function.__name__}(test_array())', number=2000, globals=globals())


def print_sorting(sorting_function):
    print(f'{sorting_function.__name__} ({time(sorting_function)} sec)')
    a = test_array()
    print(a)
    assert not is_sorted(a)
    sorting_function(a)
    assert is_sorted(a)
    print(a)

## [Bubble sort](#bubble_sort)

In [362]:
def bubble_sort(arr):
    while not is_sorted(arr):
        for a, b in bad_pairs(arr):
            swap(arr, idx(a), idx(b))

## [Quick sort](#quick_sort)

### Partition scheme

All magic of sorting happens during partition process. We divide the array on two partitions: `[low .. part] <= [part .. high]`.

There are two most popular schemes of partition: Hoare and Lomuto. Lomuto's scheme is less efficient, but more simple.

#### Lomuto partition scheme

We choose the last array item `arr[high]` as **pivot**.

Then, we shifting low bound of array `low += 1` and compare the item `arr[low]` with **pivot**. If `pivot < item`, we place the item after the pivot.

In [363]:
def partition_lomuto(arr, low, high):
    while low < high:
        if arr[high] < arr[low]:
            arr.insert(high, arr.pop(low))
            high -= 1
        else:
            low += 1
    return low


def partition_hoare(arr, low, high):
    pivot = arr[(low + high) // 2]
    i = low - 1
    j = high + 1
    while True:
        i += 1
        while arr[i] < pivot:
            i += 1
        j -= 1
        while pivot < arr[j]:
            j -= 1
        if i >= j:
            return j + 1
        swap(arr, i, j)

[1, 2, 3, 5, 4] 2
[1, 2, 3, 5, 4] 3


### Main function

This is Quick sort entry point. We recursively repeating partition function.

In [364]:
def quick_sort(arr, low=0, high=-1):
    high += 0 if 0 <= high else len(arr)
    if low < high:
        part = partition_hoare(arr, low, high)
        quick_sort(arr, low, part - 1)
        quick_sort(arr, part, high)

## [Testing](#testing)

In [365]:
print_sorting(bubble_sort)
print()
print_sorting(quick_sort)

bubble_sort (1.13927188299931 sec)
[16, 52, 81, 21, 70, 41, 23, 100, 49, 22, 32, 71, 20, 92, 68, 91, 35, 61, 22, 89, 41, 74, 65, 8, 91]
[8, 16, 20, 21, 22, 22, 23, 32, 35, 41, 41, 49, 52, 61, 65, 68, 70, 71, 74, 81, 89, 91, 91, 92, 100]

quick_sort (0.30330170999877737 sec)
[86, 66, 42, 68, 83, 2, 11, 72, 31, 57, 89, 23, 72, 59, 3, 28, 99, 16, 88, 34, 75, 39, 61, 43, 67]
[2, 3, 11, 16, 23, 28, 31, 34, 39, 42, 43, 57, 59, 61, 66, 67, 68, 72, 72, 75, 83, 86, 88, 89, 99]
